# Preprocessing of Corpora

The *Quixote* files were retrieved from the [website](http://www.husseinabbass.net/translator.html) of professor Hussein Abbass. The files are plain text files&mdash;one file per chapter of the two parts of the novel&mdash;and only require minor preprocessing: removal of bracketed numbers, collapsing of spaces to only one whitespace, and the replacement of special characters, such as é and ü.

The Ibsen files were retieved from [Project Gutenberg](www.gutenberg.org). Therefore, the files contain legal information that needs to be removed along with bracketed numbers, collapsing of spaces, and the replacement of special characters also. However, before doing that, the plays were splitted in 5 KB.

These operations are encapsulated in two functions, `quixote()` and `ibsen()`, respectively, within the module `preprocessing`. The functions use the relative paths to the folders containing the raw files (`Raw_Quixote` and `Raw_Ibsen`) in the subfolder `Corpora` and ouput the processed files in the folders `Proc_Quixote` and `Proc_Ibsen`.

In [1]:
from helper import preprocessing

In [2]:
preprocessing.quixote()

In [3]:
preprocessing.ibsen()